## C S 329E HW 9

# Crossvalidation and hyperparameter selection

## Akshay Prakash

For this week's homework we are going to explore the cross validation testing methodology and applying that to get error estimates on the two algorithms we've used so far:
  - Linear Regression
  - Decision Tree classification
  

In [1]:
# import the libraries! If you want to add things here for visualization, please do, 
# but only use sklearn when prompted
import pandas as pd
import numpy as np
from sklearn import tree 
from sklearn.datasets import load_iris
from sklearn.datasets import load_diabetes

# Part 1 - Calculate Generalized Error on Linear Regression with k-fold Cross Validation

## Q1.1 Load in the diabetes data set as a pandas dataframe and series.  
Documentation is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html).  Name your features dataframe (the independent variables) `df_X` and your target (the dependent variable) series `s_y`

In [2]:
df_X, s_y = load_diabetes(return_X_y=True, as_frame=True)

In [3]:
df_X.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641


In [4]:
s_y.head()

0    151.0
1     75.0
2    141.0
3    206.0
4    135.0
Name: target, dtype: float64

## Q1.2 Define a function that creates a linear least squares regression model 
This function should take in two parameters, `df_X`, and `s_y` and return the least squares regression model, $\hat{\beta}$ (using the notation from the ESLII book chapter 3 and HW2).  You can not use any libraries outside of pandas and numpy. 

In [5]:
def get_linear_regression_model( df_X, s_y ):
    b , residuals, rank, s = np.linalg.lstsq(df_X, s_y, rcond = -1)
    return b

## Q1.3 Define a function that partitions the dataframe and series data into dictionaries
This function should take in three parameters, `df_X`, `s_y`, and `k`, and returns a tuple of two dictionaries.
In both dictionaries the key is the `k` value (an integer from one to `k` inclusive).
The first dictionary will have the dataframe of the training data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable).
The second dictionary will have the series of the target data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable). Please note the targets _must match_ the same rows as the dataframe at this index, e.g, the length of the kth partition is the same for the dataframe and series.

Call that function with $k=5$ and create the dictionaries `dict_k_df_X` and `dict_k_s_y`. Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

In [6]:
def partition_data( df_X, s_y, k ):
    dict_k_df_X = {}
    dict_k_s_y = {}
    
    partition_index = np.random.randint(low = 1, high = k + 1, size = len(df_X))
    
    for k in np.unique(partition_index):
        
        dict_k_df_X[k], dict_k_s_y[k] = df_X[partition_index == k] , s_y[partition_index == k]
        
    return (dict_k_df_X, dict_k_s_y)

In [7]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )

In [8]:
partition_lengths = []
for key, vals in dict_k_df_X.items() :
    print("fold {} : dataframe contains {} rows, series contains {} rows.".format(key, len(vals), len(dict_k_s_y[key])))
    partition_lengths.append(len(vals))
  
print("\n{} rows in the original dataframe and {} total elements between all partitions.".format(len(df_X),sum(partition_lengths)))   

fold 1 : dataframe contains 93 rows, series contains 93 rows.
fold 2 : dataframe contains 82 rows, series contains 82 rows.
fold 3 : dataframe contains 87 rows, series contains 87 rows.
fold 4 : dataframe contains 92 rows, series contains 92 rows.
fold 5 : dataframe contains 88 rows, series contains 88 rows.

442 rows in the original dataframe and 442 total elements between all partitions.


## Q1.4 Define a function that calculates a regression metric
This function should accept two series of equal length $n$ numpy arrays, `s_y`, and `s_y_hat`. The metric it should calculate is the mean absolute error, $MAE = \sum\limits_{i=1}^n\frac{|{s\_y_i - {s\_y\_hat}_i}|}{n}$ 

Test your function by using the vectors:
```
x = np.array([1,2,3])
y = np.array([2,2,3])
```


In [9]:
def get_mae( s_y, s_y_hat):
    
    if len(s_y) == len(s_y_hat):
        mean_absolute_error = np.sum(np.abs(s_y - s_y_hat ) / len(s_y))
        return mean_absolute_error
    
    return None

In [10]:
# Test it 
x = np.array([1,2,3])
y = np.array([2,2,3])
get_mae(x,y)

0.3333333333333333

## Q1.5 Calculate the $MAE$ for each fold
For each fold in your dictionaries, calculate the $MAE$.  Use the partition number key as the test set, and all other partitions as the train set. 

Print the min, max, and mean $MAE$ of your 5 folds. 

In [11]:
mae = np.array([])
for k in dict_k_df_X.keys():
    
    # initialize an empty data frame and series to hold the training data and targets
    train_data = pd.DataFrame()
    train_y = pd.Series(dtype= float)
    
    # select the data and targets to be used as for testing
    test_data = dict_k_df_X[k].copy()
    #   adding an intercept column to account for the B0 (beta hat zero)
    test_data["intercept"] = np.ones(test_data.shape[0])
    test_s_y = dict_k_s_y[k]
    
    for train_fold in dict_k_df_X.keys():
        
        # select all partitions that are not equal to the test set key
        if train_fold != k:
            
            train_data = pd.concat([train_data, dict_k_df_X[train_fold]])
            train_y = pd.concat([train_y, dict_k_s_y[train_fold]])

    # add an intercept for the B0 term
    train_data['intercept'] = np.ones(train_data.shape[0])


    b = get_linear_regression_model(train_data, train_y)

    s_y_hat = np.matmul(test_data.to_numpy(), b)

    mae = np.append(mae, get_mae(s_y_hat, test_s_y))

print("The maximum, minimum, and mean mean average error(MAE) amongst the folds are as follows:\n{:.3f}, {:.3f}, {:.3f}, respectively.".format(mae.max(),mae.min(),mae.mean()))


The maximum, minimum, and mean mean average error(MAE) amongst the folds are as follows:
50.745, 41.371, 44.340, respectively.


# Part 2 - Find the best hyperparameter to use in a Decision Tree 

## Q2.1 Load the iris data in as a pandas dataframe and a series
Documentation is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). Name your features dataframe (the independent variables) `df_X` and your classification target (the dependent variable) series `s_y`

In [12]:
df_X , s_y = load_iris(return_X_y=True, as_frame=True)

## Q2.2 Partition `df_X` and `s_y` into $5$ partitions of roughly equal size
Make 2 dictionaries, with the key of each dictionary the fold number.  The value of the dictionary `dict_k_df_X` is the $k^{th}$ partition of the data, and the value of the dictionary `dict_k_s_y` is the corresponding $k^{th}$ target classification.  Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

In [13]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )

In [14]:
partition_lengths = []
for key, vals in dict_k_df_X.items() :
    print("fold {} : dataframe contains {} rows, series contains {} rows.".format(key, len(vals), len(dict_k_s_y[key])))
    partition_lengths.append(len(vals))
  
print("\n{} rows in the original dataframe and {} total elements between all partitions.".format(len(df_X),sum(partition_lengths)))   

fold 1 : dataframe contains 29 rows, series contains 29 rows.
fold 2 : dataframe contains 35 rows, series contains 35 rows.
fold 3 : dataframe contains 28 rows, series contains 28 rows.
fold 4 : dataframe contains 34 rows, series contains 34 rows.
fold 5 : dataframe contains 24 rows, series contains 24 rows.

150 rows in the original dataframe and 150 total elements between all partitions.


## Q2.3 Define a function that calculates accuracy
The function should accept two series and compare each element for equality.  The accuracy is the number of equal elements divided by the total number of elements.

Test your accuracy function by calling it with the `s_y` loaded from the iris data set and an array of the same length containing all $1$ values. 

In [15]:
def get_acc( s_1, s_2 ):
    if s_1.size == s_2.size:
        return np.sum(s_1 == s_2) / s_1.size

In [16]:
get_acc(s_y,np.ones(len(s_y)))

0.3333333333333333

## Q2.4 Using Nested Cross validation, find the best hyperparameter
Use the [Decision Tree Classifier](https://scikit-learn.org/stable/modules/tree.html#classification) class to build a decision tree inside of a 5-fold cross validation loop.  The partitions you already created in 2.2 will be the outer loop.  In the inside loop you should use 4-fold cross validation (so you don't have to partition _again_) to find the best value for `min_impurity_decrease`.  Use the Gini Index as your impurity measure. 
    Calculate the mean accuracy across the 4 folds of your inner loop for all the candidate `min_impurity_decrease` values, and print the value.  Use the array `np.array([0.1,0.25,0.3,0.4])` as the candidates for the best hyperparameter. If there is a tie (two `min_impurity_decrease` values give the same highest accuracy), choose the lowest `min_impurity_decrease` value. 

For each inner loop, select the best `min_impurity_decrease` and train the outer fold training data on using that value. 

For each inner loop, your output should look something like this:
```
Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 0.95
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.86
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 0.63
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 0.27

Best min impurity decrease is 0.1

```

In [21]:
possible_min_impurity_decrease = np.array([0.1,0.25,0.3,0.4])

# Outer loop
outer_acc = np.array([])
for k in dict_k_df_X.keys():
    
    train_df_X = pd.DataFrame()
    train_s_y = pd.Series(dtype=float)
    
    average_pos_min_impurity_accuracy = np.array([])
    
    
    for pos_min_impurity in possible_min_impurity_decrease:
        
        print("Testing {:.2f} min impurity decrease".format(pos_min_impurity))
        
        pos_min_impurity_acc = np.array([])
        keys_list = list(dict_k_df_X.keys())
        keys_list.remove(k)
        
        # Inner loop cross validation code here (use 4 folds, where the fold does not include k)
        
        for inner_k in keys_list:
            
            inner_training_data = pd.DataFrame()
            inner_training_y = pd.Series(dtype= float)
            
            inner_test_data = dict_k_df_X[inner_k]
            inner_test_y = dict_k_s_y[inner_k]
            
            for not_inner_k in keys_list:
                
                if not_inner_k != inner_k:
                    inner_training_data = pd.concat([inner_training_data,dict_k_df_X[not_inner_k]])
                    inner_training_y = pd.concat([inner_training_y , dict_k_s_y[not_inner_k]])
            
            clf = tree.DecisionTreeClassifier(criterion= 'gini', min_impurity_decrease= pos_min_impurity)
            clf.fit(inner_training_data, inner_training_y)
            
            current_accuracy = get_acc(inner_test_y, clf.predict(inner_test_data))
            pos_min_impurity_acc = np.append(pos_min_impurity_acc, current_accuracy)
            
        average_pos_min_impurity_accuracy = np.append(average_pos_min_impurity_accuracy , pos_min_impurity_acc.mean())
        
        print("\tAverage Accuracy over {:d} folds is {:.2f}".format(len(keys_list), pos_min_impurity_acc.mean()))
        
    index_highest_accuracy = np.argmin(1-average_pos_min_impurity_accuracy)
    print("\nBest minimum impurity decrease value is {}\n".format(possible_min_impurity_decrease[index_highest_accuracy]))
        
    current_min_impurity_decrease = possible_min_impurity_decrease[index_highest_accuracy]
            
    for not_k in dict_k_df_X.keys():
            
        if not_k != k:
                
            train_df_X = pd.concat([train_df_X, dict_k_df_X[not_k]])
            train_s_y = pd.concat([train_s_y, dict_k_s_y[not_k]])
        
    test_data = dict_k_df_X[k]
    test_s_y = dict_k_s_y[k]
        
    clf = tree.DecisionTreeClassifier(criterion='gini', min_impurity_decrease= current_min_impurity_decrease)
    clf = clf.fit(train_df_X, train_s_y)  
        
    
    current_accuracy_outer = get_acc(test_s_y, clf.predict(test_data))
    outer_acc = np.append(outer_acc,current_accuracy_outer) # make sure and calculate this_acc in your loop

Testing 0.10 min impurity decrease
	Average Accuracy over 4 folds is 0.93
Testing 0.25 min impurity decrease
	Average Accuracy over 4 folds is 0.86
Testing 0.30 min impurity decrease
	Average Accuracy over 4 folds is 0.71
Testing 0.40 min impurity decrease
	Average Accuracy over 4 folds is 0.30

Best minimum impurity decrease value is 0.1

Testing 0.10 min impurity decrease
	Average Accuracy over 4 folds is 0.92
Testing 0.25 min impurity decrease
	Average Accuracy over 4 folds is 0.81
Testing 0.30 min impurity decrease
	Average Accuracy over 4 folds is 0.54
Testing 0.40 min impurity decrease
	Average Accuracy over 4 folds is 0.25

Best minimum impurity decrease value is 0.1

Testing 0.10 min impurity decrease
	Average Accuracy over 4 folds is 0.92
Testing 0.25 min impurity decrease
	Average Accuracy over 4 folds is 0.67
Testing 0.30 min impurity decrease
	Average Accuracy over 4 folds is 0.63
Testing 0.40 min impurity decrease
	Average Accuracy over 4 folds is 0.30

Best minimum impuri

## Q2.5 Show the generalized performance of the classifier 
Show the generalized performance of the classifier by printing the min, max, and mean accuracy of the outer fold test sets.

In [22]:
print("The minimum accuracy is {:.2f}, the maximum accuracy is {:.2f}, and the mean accuracy is {:.2f}".format(outer_acc.min(), outer_acc.max(), outer_acc.mean()))

The minimum accuracy is 0.83, the maximum accuracy is 0.96, and the mean accuracy is 0.92
